In [2]:
from __future__ import print_function, division
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import LSTM, GRU, SimpleRNN, Dense, Input, BatchNormalization
from keras.utils import get_file
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Nadam
from sklearn import preprocessing
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from collections import Counter
import numpy as np
import copy
import csv
from sklearn import metrics
import time
from datetime import datetime
from math import log
import pandas as pd
pd.set_option('display.max_rows', None)

from datetime import datetime
import time

model = load_model(r"output_files\models\model_22-0.92_IoT_cols25.h5") # load model, set this to the model generated by train.py
filename = 'event_data_aggregated_alex'
df = pd.read_csv('data/' + filename + '_reworked.csv')
print('the shape of the data', df.shape)

# these are just the variables we need to initialize
# Create lists dynamically for each group of columns
IoT_seqs = {col: [] for col in df.columns if col not in ['CaseID', 'ActivityID', 'CompleteTimestamp']} # the IoT_seqs is a dictionary of column values(keys) and the values are the cases. Each case is a trace of length between 0 and 44
IoT_lines = {col: [] for col in df.columns if col not in ['CaseID', 'ActivityID', 'CompleteTimestamp']}
lineseq, timeseqs, timeseqs2, timeseqs3, timeseqs4 = [], [], [], [], []
lastcase, line, firstLine = '', '', True
lines, times, times2, numlines, index = [], [], [], 0, 0
casestarttime, lasteventtime = None, None
step, sentences, softness, next_chars = 1, [], 0, [] # Vectorization
sentences_t, sentences_t2, sentences_t3, sentences_t4 = [], [], [], []
next_chars_t, next_chars_t2, next_chars_t3, next_chars_t4 = [], [], [], []
caseids = []

#########################################################################################
#the next code snippet basically creates lists of lists with all the relevant information
#########################################################################################

eventlog = filename + '_reworked'
csvfile = open('data/%s' % eventlog+'.csv', 'r')
datareader = csv.reader(csvfile, delimiter=',', quotechar='|') # Load data from CSV file
next(datareader, None)  # skip the headers
for row in datareader: #the columns are "CaseID, ActivityID, CompleteTimestamp"
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S") # creates a datetime object from row[2]
    if row[0] != lastcase:  #'lastcase' is to save the last executed case for the loop. We only go in this if-statement if we have a new case
        casestarttime = t 
        lasteventtime = t
        lastcase = row[0]
        if not firstLine: # we always do this if-statement, only not for the first ever line (would add an empty list to a list)
            lineseq.append(lines)
            timeseqs.append(times)
            timeseqs2.append(times2)
            timeseqs3.append(times3)
            timeseqs4.append(times4)
            for index, (col, values) in enumerate(IoT_seqs.items()):
                IoT_seqs[col].append(IoT_lines[col])
        IoT_lines = {col: [] for col in df.columns if col not in ['CaseID', 'ActivityID', 'CompleteTimestamp']}
        lines = []
        times = []
        times2 = []
        times3 = []
        times4 = []
        
        numlines+=1
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    midnight = datetime.fromtimestamp(time.mktime(t)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(t))-midnight
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds     #multiply with 60*60*24 = 86400 to go from days to seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds    #the .seconds method gives the time in seconds
    timediff3 = timesincemidnight.seconds #this leaves only time event occured after midnight
    timediff4 = datetime.fromtimestamp(time.mktime(t)).weekday() #day of the week
    caseids.append(row[0])
    lines.append(str(row[1])) #add the activity label to the line list
    times.append(timediff)
    times2.append(timediff2)
    times3.append(datetime.fromtimestamp(time.mktime(t)))
    times4.append(timediff4)
    for index, (col, values) in enumerate(IoT_lines.items()):
        IoT_lines[col].append(str(row[3 + index]))
    lasteventtime = t
    index = index+1
    firstLine = False #after the first line we set the FirstLine to False; this is because we want to save the lines and times lists to the lineseq and timeseq list for each seperate case

# add last case
lineseq.append(lines)
timeseqs.append(times)
timeseqs2.append(times2)
timeseqs3.append(times3)
timeseqs4.append(times4)
for index, (col, values) in enumerate(IoT_seqs.items()):
    IoT_seqs[col].append(IoT_lines[col])
numlines+=1

# Calculate divisors
divisor = np.mean([item for sublist in timeseqs for item in sublist]) # the average time difference between the event in a case and the previous event, across all events in the dataset
divisor2 = np.mean([item for sublist in timeseqs2 for item in sublist]) # average time difference since the start of a case across all events in the dataset.

print('divisor: {}'.format(divisor))
print('divisor2: {}'.format(divisor2))

maxlen = max(map(lambda x: len(x),lineseq))
chars = map(lambda x : set(x),lineseq)
chars = list(set().union(*chars))
target_chars = copy.copy(chars)

# Custom sorting function to convert strings to integers for numerical sorting
def custom_sort(item):
    return int(item)

# Sorting the list using the custom sorting function (because we sort strings based on their integer value)
chars = sorted(chars, key=custom_sort)

print('total chars: {}, target chars: {}'.format(len(chars), len(target_chars)))
print('maxlen', maxlen)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
target_char_indices = dict((c, i) for i, c in enumerate(target_chars))
target_indices_char = dict((i, c) for i, c in enumerate(target_chars))

predict_size = 1 # set parameters

def getSymbol(predictions):
    maxPrediction = 0
    symbol = ''
    i = 0
    for prediction in predictions:
        if(prediction>=maxPrediction):
            maxPrediction = prediction
            symbol = target_indices_char[i]
        i += 1
    return symbol

def encode(sentence, times, times3, IoT_seqs, counter, maxlen=maxlen):
    num_features = len(chars)+5 + len(IoT_seqs.keys()) 
    X = np.zeros((1, maxlen, num_features), dtype=np.float32)
    leftpad = maxlen-len(sentence)
    times2 = np.cumsum(times)
    for t, char in enumerate(sentence):
        midnight = times3[t].replace(hour=0, minute=0, second=0, microsecond=0)
        timesincemidnight = times3[t]-midnight
        for c in chars:
            if c==char:
                X[0, t+leftpad, char_indices[c]] = 1
        X[0, t+leftpad, len(chars)] = t+1
        X[0, t+leftpad, len(chars)+1] = times[t]/divisor
        X[0, t+leftpad, len(chars)+2] = times2[t]/divisor2
        X[0, t+leftpad, len(chars)+3] = timesincemidnight.seconds/86400
        X[0, t+leftpad, len(chars)+4] = times3[t].weekday()/7

        # Add information for IoT columns
        for col in IoT_seqs.keys():
            X[0, t + leftpad, len(chars) + 5 + list(IoT_seqs.keys()).index(col)] = IoT_seqs[col][counter][t]

    return X

# make predictions
correct = 0
counter_total = 0
with open('output_files/results/next_activity_and_time_%s' % eventlog, 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(["CaseID", "Prefix length", "Groud truth", "Predicted", "MAE"])
    for prefix_size in range(2,maxlen):
        print('prefix size', prefix_size)
        for counter, (line, caseid, times, times3) in enumerate(zip(lineseq, caseids, timeseqs, timeseqs3)):
            print('original trace', line)
            times.append(0)
            cropped_line = line[:prefix_size]
            cropped_times = times[:prefix_size]
            cropped_times3 = times3[:prefix_size]
            print('cropped line', cropped_line)

            ground_truth = line[prefix_size:prefix_size+predict_size]
            for i in range(predict_size):
                if len(ground_truth)<=i:
                    continue
                counter_total +=1
                enc = encode(cropped_line, cropped_times, cropped_times3, IoT_seqs, counter)
                print(enc.shape)

                y = model.predict(enc, verbose=0)
                y_char = y[0]
                prediction = np.argmax(y_char).item() 
                print('ground truth', int(ground_truth[0]))
                print('prediction', prediction)
                if prediction == int(ground_truth[0]):
                    correct +=1
                if correct >0:
                    print('accuracy:', correct/counter_total)



the shape of the data (3712, 28)
divisor: 21512.25484913793
divisor2: 242251.2327586207
total chars: 15, target chars: 15
maxlen 44
prefix size 2
original trace ['9', '0', '13', '12', '14', '13', '14', '8', '6', '3', '13', '14', '13', '4', '14', '1', '3', '13', '14', '13', '4', '2', '12', '7', '5', '13', '14', '14', '13', '13', '14', '11']
cropped line ['9', '0']
(1, 44, 45)
ground truth 13
prediction 0
original trace ['9', '0', '12', '12', '13', '12', '14', '13', '14', '8', '3', '6', '13', '4', '2', '14', '7', '14', '5', '13', '14', '13', '12', '14', '11']
cropped line ['9', '0']
(1, 44, 45)
ground truth 12
prediction 0
original trace ['9', '0', '13', '14', '8', '6', '3', '13', '14', '13', '4', '2', '14', '13', '12', '7', '14', '5', '13', '14', '13', '14', '11']
cropped line ['9', '0']
(1, 44, 45)
ground truth 13
prediction 0
original trace ['6', '3', '13', '4', '2', '14', '12', '13', '7', '5']
cropped line ['6', '3']
(1, 44, 45)
ground truth 13
prediction 0
original trace ['3', '6'